# Social Computing (CS60017) Autumn 2021
## Assignment 2: Network centrality and community structure
## **Question 1**
**Pratyush Jaiswal  
18EE35014**

## **Importing**

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from pathlib import Path
from networkx.algorithms import community
from collections import defaultdict
import os
from config import CONFIG

## **Configuration**

In [ ]:
# Configuring file locations of datasets, plots and output files
# Datasets
FOOTBALL_DATASET = os.path.join(CONFIG["DATASET_DIR"], "football.gml")
POLBOOKS_DATASET = os.path.join(CONFIG["DATASET_DIR"], "polbooks.gml")

FOOTBALL_TRUE_COMMS_NUM = 12
POLBOOKS_TRUE_COMMS_NUM = 3

# Plot Directory
PLOT_DIR = CONFIG["PLOT_DIR"]

# Output Text Files
NEWMAN_OUTPUT_FILE = Path(".\output_1_NEWMAN.txt")
CLAUSET_OUTPUT_FILE = Path(".\output_1_CLAUSET.txt")

# Making the blank output files
with open(NEWMAN_OUTPUT_FILE, "w") as f:
    f.write("")
with open(CLAUSET_OUTPUT_FILE, "w") as f:
    f.write("")

newman_file = None
clauset_file = None

NEWMAN_METHOD_NAME = "NEWMAN"
CLAUSET_METHOD_NAME = "CLAUSET"
FOOTBALL_DATASET_NAME = "FOOTBALL"
POLBOOKS_DATASET_NAME = "POLBOOKS"

### Functions for writing outputs to files

In [ ]:
def open_NEWMAN():
    global newman_file
    newman_file = open(NEWMAN_OUTPUT_FILE, "a")

def close_NEWMAN():
    global newman_file
    newman_file.close()

def open_CLAUSET():
    global clauset_file
    clauset_file = open(CLAUSET_OUTPUT_FILE, "a")

def close_CLAUSET():
    global clauset_file
    clauset_file.close()

## **Reading Graphs from Datasets**

In [ ]:
# Reading graphs
FOOTBALL_GRAPH = nx.read_gml(FOOTBALL_DATASET)
POLBOOKS_GRAPH = nx.read_gml(POLBOOKS_DATASET)

## **Part [a]**

### Function to get the communities list for given datasets using the given two methods

In [ ]:
def get_num_of_communities(methodname, datasetname, graph, output_file, true_comms_num=None):
    '''
    Calculates the number of communities generated

    Also returns the list of communities
    '''
    print("Dataset: {}".format(datasetname), file=output_file)
    if(methodname == NEWMAN_METHOD_NAME):
        communities = list(list(community.girvan_newman(graph))[true_comms_num-2])
        num_communities = len(communities)
        print("Number of communities using Newman-Girvan method: {}".format(num_communities), file=output_file)
        
    elif(methodname == CLAUSET_METHOD_NAME):
        communities = list(community.greedy_modularity_communities(graph))
        num_communities = len(communities)
        print("Number of communities using Clauset-Newman-Moore greedy modularity maximization method: {}".format(num_communities), file=output_file)
    print(file=output_file)
    return communities

In [ ]:
open_NEWMAN()
FOOTBALL_NEWMAN_COMMUNITIES = get_num_of_communities(NEWMAN_METHOD_NAME, FOOTBALL_DATASET_NAME, FOOTBALL_GRAPH, newman_file, FOOTBALL_TRUE_COMMS_NUM)
POLBOOKS_NEWMAN_COMMUNITIES = get_num_of_communities(NEWMAN_METHOD_NAME, POLBOOKS_DATASET_NAME, POLBOOKS_GRAPH, newman_file, POLBOOKS_TRUE_COMMS_NUM)
close_NEWMAN()
open_CLAUSET()
FOOTBALL_CLAUSET_COMMUNITIES = get_num_of_communities(CLAUSET_METHOD_NAME, FOOTBALL_DATASET_NAME, FOOTBALL_GRAPH, clauset_file, FOOTBALL_TRUE_COMMS_NUM)
POLBOOKS_CLAUSET_COMMUNITIES = get_num_of_communities(CLAUSET_METHOD_NAME, POLBOOKS_DATASET_NAME, POLBOOKS_GRAPH, clauset_file, POLBOOKS_TRUE_COMMS_NUM)
close_CLAUSET()

## **Part [b]**

### Function to plot the communtiy size distribution 

In [ ]:
def plot_community_size_distribution(methodname, datasetname, communities):
    '''
    Plots the community size distribution for diff datasets and methods

    communities: List of communities
    '''
    print("Plotting Community Size distribution of {} graph using {} method".format(datasetname, methodname))
    points = []
    for c in communities:
        points.append(len(c))
    plt.hist(points, rwidth=0.2)
    plt.xlabel("Size of Communities")
    plt.ylabel("Frequency")
    plt.title("Community Size distribution of {} graph using {} method".format(datasetname, methodname))
    fig_destination = os.path.join(PLOT_DIR, f"{datasetname}_{methodname}_dist.png")
    plt.savefig(fig_destination)
    plt.close()
    print("Plotting Done")

In [ ]:
plot_community_size_distribution(NEWMAN_METHOD_NAME, FOOTBALL_DATASET_NAME, FOOTBALL_NEWMAN_COMMUNITIES)
plot_community_size_distribution(NEWMAN_METHOD_NAME, POLBOOKS_DATASET_NAME, POLBOOKS_NEWMAN_COMMUNITIES)
plot_community_size_distribution(CLAUSET_METHOD_NAME, FOOTBALL_DATASET_NAME, FOOTBALL_CLAUSET_COMMUNITIES)
plot_community_size_distribution(CLAUSET_METHOD_NAME, POLBOOKS_DATASET_NAME, POLBOOKS_CLAUSET_COMMUNITIES)

## **Part [c]**

### Function to plot the ground truth community size distribution and also returns the ground truth communities list

In [ ]:
def plot_ground_truth_community_size_distribution(methodname, datasetname, graph):
    '''
    Plots the ground truth community size distribution
    Also returns the ground truth community list
    
    Here the nodes with same `value` are being considered in the same true community.
    '''
    # Making the ground truth community
    communities = defaultdict(list)
    for node, value in list(graph.nodes(data="value")):
        communities[value].append(node)
    ground_truth_communities = list(communities.values())
    
    # Plotting the histogram
    print("Plotting Ground Truth Community Size distribution of {} using {} method".format(datasetname, methodname))
    points = []
    for c in ground_truth_communities:
        points.append(len(c))
    plt.hist(points, rwidth=0.2)
    plt.xlabel("Size of Ground Truth Communities")
    plt.ylabel("Frequency")
    plt.title("Ground Truth Community Size distribution of {} using {} method".format(datasetname, methodname))
    fig_destination = os.path.join(PLOT_DIR, f"ground_truth_communities_dist_{datasetname}_{methodname}.png")
    plt.savefig(fig_destination)
    plt.close()
    print("Plotting Done")
    return ground_truth_communities

In [ ]:
FOOTBALL_NEWMAN_GROUND_TRUTH_COMMUNITIES = plot_ground_truth_community_size_distribution(NEWMAN_METHOD_NAME, FOOTBALL_DATASET_NAME, FOOTBALL_GRAPH)
POLBOOKS_NEWMAN_GROUND_TRUTH_COMMUNITIES = plot_ground_truth_community_size_distribution(NEWMAN_METHOD_NAME, POLBOOKS_DATASET_NAME, POLBOOKS_GRAPH)
FOOTBALL_CLAUSET_GROUND_TRUTH_COMMUNITIES = plot_ground_truth_community_size_distribution(CLAUSET_METHOD_NAME, FOOTBALL_DATASET_NAME, FOOTBALL_GRAPH)
POLBOOKS_CLAUSET_GROUND_TRUTH_COMMUNITIES = plot_ground_truth_community_size_distribution(CLAUSET_METHOD_NAME, POLBOOKS_DATASET_NAME, POLBOOKS_GRAPH)

## **Part [d]**

In [ ]:
def get_top_5_communities(communities):
    '''
    Get the top 5 communities based on the number of nodes from a list of communities
    '''
    return sorted(communities, key=lambda com: len(com), reverse=True)[:5]

In [ ]:
FOOTBALL_NEWMAN_TOP_5_COMMUNITIES = get_top_5_communities(FOOTBALL_NEWMAN_COMMUNITIES)
POLBOOKS_NEWMAN_TOP_5_COMMUNITIES = get_top_5_communities(POLBOOKS_NEWMAN_COMMUNITIES)
FOOTBALL_CLAUSET_TOP_5_COMMUNITIES = get_top_5_communities(FOOTBALL_CLAUSET_COMMUNITIES)
POLBOOKS_CLAUSET_TOP_5_COMMUNITIES = get_top_5_communities(POLBOOKS_CLAUSET_COMMUNITIES)

### Function to generate subgraphs from a list of communities

In [ ]:
def get_subragphs_from_communities(graph, communities):
    '''
    Get a list of subgraphs from a list of community nodes and the original graph
    '''
    communities_subgraphs = []
    for com in communities:
        subgraph = graph.subgraph(com).copy()
        communities_subgraphs.append(subgraph)
    return communities_subgraphs

In [ ]:
FOOTBALL_NEWMAN_TOP_5_COMMUNITIES_SUBGRAPHS = get_subragphs_from_communities(FOOTBALL_GRAPH, FOOTBALL_NEWMAN_TOP_5_COMMUNITIES)
POLBOOKS_NEWMAN_TOP_5_COMMUNITIES_SUBGRAPHS = get_subragphs_from_communities(POLBOOKS_GRAPH, POLBOOKS_NEWMAN_TOP_5_COMMUNITIES)
FOOTBALL_CLAUSET_TOP_5_COMMUNITIES_SUBGRAPHS =  get_subragphs_from_communities(FOOTBALL_GRAPH, FOOTBALL_CLAUSET_TOP_5_COMMUNITIES)
POLBOOKS_CLAUSET_TOP_5_COMMUNITIES_SUBGRAPHS = get_subragphs_from_communities(POLBOOKS_GRAPH, POLBOOKS_CLAUSET_TOP_5_COMMUNITIES)

### Function to print the size (num_nodes, num_edges) of top 5 communities

In [ ]:
def print_top_5_communities_size(methodname, datasetname, community_subgraphs, output_file):
    '''
    Print the size of the top 5 communities in the form of (number of nodes, number of edges)
    '''
    print("Method: {}".format(methodname), file=output_file)
    print("Dataset: {}".format(datasetname), file=output_file)
    for i in range(len(community_subgraphs)):
        community_subgraph = community_subgraphs[i]
        number_of_nodes = community_subgraph.number_of_nodes()
        number_of_edges = community_subgraph.number_of_edges()
        print(f"Community {i+1}: (number_of_nodes = {number_of_nodes}, number_of_edges = {number_of_edges})", file=output_file)
    print(file=output_file)

In [ ]:
open_NEWMAN()
print_top_5_communities_size(NEWMAN_METHOD_NAME, FOOTBALL_DATASET_NAME, FOOTBALL_NEWMAN_TOP_5_COMMUNITIES_SUBGRAPHS, newman_file)
print_top_5_communities_size(NEWMAN_METHOD_NAME, POLBOOKS_DATASET_NAME, POLBOOKS_NEWMAN_TOP_5_COMMUNITIES_SUBGRAPHS, newman_file)
close_NEWMAN()
open_CLAUSET()
print_top_5_communities_size(CLAUSET_METHOD_NAME, FOOTBALL_DATASET_NAME, FOOTBALL_CLAUSET_TOP_5_COMMUNITIES_SUBGRAPHS, clauset_file)
print_top_5_communities_size(CLAUSET_METHOD_NAME, POLBOOKS_DATASET_NAME, POLBOOKS_CLAUSET_TOP_5_COMMUNITIES_SUBGRAPHS, clauset_file)
close_CLAUSET()

## **Part [e]**

### Function to calculate and print the coverage of communities using the communities subgraphs (calculated above)

In [ ]:
def get_top_5_community_coverage(methodname, datasetname, graph, coummunities_subgraphs, output_file):
    '''
    Get the coverage of the top 5 communities
    '''
    print("Method: {}".format(methodname), file=output_file)
    print("Dataset: {}".format(datasetname), file=output_file)
    for i in range(len(coummunities_subgraphs)):
        # Using the formula for coverage = number_of_intra_community_edges / tot_edges_in_graph
        coverage = coummunities_subgraphs[i].number_of_edges() / graph.number_of_edges()
        print("Coverage of community {} = {}".format(i+1, coverage), file=output_file)
    print(file=output_file)

In [ ]:
open_NEWMAN()
get_top_5_community_coverage(NEWMAN_METHOD_NAME, FOOTBALL_DATASET_NAME, FOOTBALL_GRAPH, FOOTBALL_NEWMAN_TOP_5_COMMUNITIES_SUBGRAPHS, newman_file)
get_top_5_community_coverage(NEWMAN_METHOD_NAME, POLBOOKS_DATASET_NAME, POLBOOKS_GRAPH, POLBOOKS_NEWMAN_TOP_5_COMMUNITIES_SUBGRAPHS, newman_file)
close_NEWMAN()
open_CLAUSET()
get_top_5_community_coverage(CLAUSET_METHOD_NAME, FOOTBALL_DATASET_NAME, FOOTBALL_GRAPH, FOOTBALL_CLAUSET_TOP_5_COMMUNITIES_SUBGRAPHS, clauset_file)
get_top_5_community_coverage(CLAUSET_METHOD_NAME, POLBOOKS_DATASET_NAME, POLBOOKS_GRAPH, POLBOOKS_CLAUSET_TOP_5_COMMUNITIES_SUBGRAPHS, clauset_file)
close_CLAUSET()

## **Part [f]**

### Function to calculate the jaccard coefficients between two communities

In [ ]:
def get_jaccard_coefficient(methodname, datasetname, community1, community2, output_file):
    '''
    Calculates the jaccard coefficient between two communities
    '''
    print("Method: {}".format(methodname), file=output_file)
    print("Dataset: {}".format(datasetname), file=output_file)
    union_size = len(set(community1) | set(community2))
    intersection_size = len(set(community1) & set(community2))
    coeff = 0
    if(union_size != 0):
        coeff = intersection_size / union_size
    print("jaccard coefficient = {}".format(coeff), file=output_file)
    print(file=output_file)

### Function to get the top communtiy based on the number of nodes

In [ ]:
def get_top_communtiy(communities):
    '''
    Get the top community based on the number of nodes from a list of communities
    '''
    return sorted(communities, key=lambda com: len(com), reverse=True)[0]

In [ ]:
FOOTBALL_NEWMAN_TOP_COMMUNITY = FOOTBALL_NEWMAN_TOP_5_COMMUNITIES[0]
POLBOOKS_NEWMAN_TOP_COMMUNITY = POLBOOKS_NEWMAN_TOP_5_COMMUNITIES[0]
FOOTBALL_CLAUSET_TOP_COMMUNITY = FOOTBALL_CLAUSET_TOP_5_COMMUNITIES[0]
POLBOOKS_CLAUSET_TOP_COMMUNITY = POLBOOKS_CLAUSET_TOP_5_COMMUNITIES[0]

In [ ]:

FOOTBALL_NEWMAN_GROUND_TRUTH_TOP_COMMUNITY = get_top_communtiy(FOOTBALL_NEWMAN_GROUND_TRUTH_COMMUNITIES)
POLBOOKS_NEWMAN_GROUND_TRUTH_TOP_COMMUNITY = get_top_communtiy(POLBOOKS_NEWMAN_GROUND_TRUTH_COMMUNITIES)
FOOTBALL_CLAUSET_GROUND_TRUTH_TOP_COMMUNITY = get_top_communtiy(FOOTBALL_CLAUSET_GROUND_TRUTH_COMMUNITIES)
POLBOOKS_CLAUSET_GROUND_TRUTH_TOP_COMMUNITY = get_top_communtiy(POLBOOKS_CLAUSET_GROUND_TRUTH_COMMUNITIES)

In [ ]:
open_NEWMAN()
get_jaccard_coefficient(NEWMAN_METHOD_NAME, FOOTBALL_DATASET_NAME, FOOTBALL_NEWMAN_TOP_COMMUNITY, FOOTBALL_NEWMAN_GROUND_TRUTH_TOP_COMMUNITY, newman_file)
get_jaccard_coefficient(NEWMAN_METHOD_NAME, POLBOOKS_DATASET_NAME, POLBOOKS_NEWMAN_TOP_COMMUNITY, POLBOOKS_NEWMAN_GROUND_TRUTH_TOP_COMMUNITY, newman_file)
close_NEWMAN()
open_CLAUSET()
get_jaccard_coefficient(CLAUSET_METHOD_NAME, FOOTBALL_DATASET_NAME, FOOTBALL_CLAUSET_TOP_COMMUNITY, FOOTBALL_CLAUSET_GROUND_TRUTH_TOP_COMMUNITY, clauset_file)
get_jaccard_coefficient(CLAUSET_METHOD_NAME, POLBOOKS_DATASET_NAME, POLBOOKS_CLAUSET_TOP_COMMUNITY, POLBOOKS_CLAUSET_GROUND_TRUTH_TOP_COMMUNITY, clauset_file)
close_CLAUSET()